In [ ]:
# Sample Code for Final Project
# Sarah Brannum

In [ ]:
# import packages
import os #to read the file path
import numpy as np
from matplotlib import pyplot as plt
import xarray as xr # to read in the netcdf file Delft3D output
from scipy.spatial import cKDTree
from scipy import stats
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
from pyproj import Proj
import pandas as pd
from matplotlib.ticker import FormatStrFormatter
import contextily as ctx
import cmocean # for specific color scale
import dfm_tools as dfmt # Delft3D package for reading in codes
from datetime import datetime, timedelta # to convert datetime objects into different units

In [ ]:
# This sections defines the Delft3D output file path, and brings it into jupyter as an xarray

#file_location=r"C:\Users\sbrann5\OneDrive - Louisiana State University\3_Western_Scheldt\02_Final_models_incl_output\Western_Scheldt_d.dsproj_data\FlowFM\output\FlowFM_map.nc"
file_location=r"C:\Users\sbrann5\Downloads\FlowFM_map_100.nc"
map_xr = xr.open_dataset(file_location)
map_xr.rename({})


In [ ]:
# read in netcdf file using the dfmt 
file_nc = os.path.join(file_location)
uds_map = dfmt.open_partitioned_dataset(file_nc)
uds_map = uds_map.rename({})
map_vars_pd = dfmt.get_ncvarproperties(uds_map)


In [ ]:
# Building a Pandas dataframe that writes all the output variables given by Delft3D, where they are defined, and type of data

nc_varkeys = map_xr.variables.mapping.keys() 
list_varattrs_pd = []
for varkey in nc_varkeys:
    varattrs_pd = pd.DataFrame({varkey:map_xr.variables.mapping[varkey].attrs}).T
    varattrs_pd[['shape','dimensions','dtype']] = 3*[''] 
    varattrs_pd.at[varkey,'shape'] = map_xr[varkey].shape
    varattrs_pd.at[varkey,'dimensions'] = map_xr.variables[varkey].dims
    varattrs_pd.at[varkey,'dtype'] = map_xr.variables[varkey].dtype
    list_varattrs_pd.append(varattrs_pd)
vars_pd = pd.concat(list_varattrs_pd,axis=0)
vars_pd[vars_pd.isnull()] = ''
vars_pd = vars_pd[["standard_name","units","location","shape","dimensions","dtype","long_name"]]

print(vars_pd)
vars_pd.to_csv(r'C:\Users\sbrann5\OneDrive - Louisiana State University\May13_backup\Classwork\python_course\final_proj\variables.csv', encoding='utf-8', index=False, header=True)

In [ ]:
# Set figure dimensions
fig_width = 10
fig_height = 4
font_size = 12

In [ ]:
# Plot grid with background of the area

fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=600)
uds_map.grid.plot(edgecolor='crimson', linewidth=0.5)
ctx.add_basemap(ax, crs='EPSG:28992', source=ctx.providers.CartoDB.VoyagerNoLabels, alpha = 0.75) # Alpha changes the transparency of the basemap
# Above is the basemap. Here are other basemap options (https://contextily.readthedocs.io/en/latest/intro_guide.html)
ax.set_aspect('equal')
ax.set_xlabel('Longitude', fontsize=font_size)
ax.set_ylabel('Latitude', fontsize=font_size)
plt.title('Grid and Model Domain') 
plt.savefig(r'C:\Users\sbrann5\OneDrive - Louisiana State University\May13_backup\Classwork\python_course\final_proj\figures\grid_and_model_domain.png', bbox_inches='tight') # use bbox to remove white space around image
plt.show()

In [ ]:
# find minimum and maximum depth for plotting depth

min_depth=np.min(uds_map['mesh2d_flowelem_bl'])
print(min_depth.values)
max_depth=np.max(uds_map['mesh2d_flowelem_bl'])
print(max_depth.values)

In [ ]:
# Depth
fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=600)
pc = uds_map['mesh2d_flowelem_bl'].ugrid.plot(cmap=cmocean.cm.deep, add_colorbar=False, ax=ax, vmin=min_depth.values, vmax=max_depth.values)
#ctx.add_basemap(ax, crs='EPSG:28992', source=ctx.providers.CartoDB.VoyagerNoLabels, alpha = 0.75) # Alpha changes the transparency of the basemap
# Above is the basemap. Here are other basemap options (https://contextily.readthedocs.io/en/latest/intro_guide.html)
cbar = fig.colorbar(pc, ax=ax)
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cbar.set_label('Elevation [m]', fontsize=font_size)
ax.set_aspect('equal')
ax.set_facecolor('whitesmoke')
ax.set_xlabel('Longitude', fontsize=font_size)
ax.set_ylabel('Latitude', fontsize=font_size)
plt.title('Elevation in the Western Scheldt') 
plt.savefig(r'C:\Users\sbrann5\OneDrive - Louisiana State University\May13_backup\Classwork\python_course\final_proj\figures\initial_depth.png', bbox_inches='tight') # use bbox to remove white space around image
plt.show()

In [ ]:
# Velocity at a specific time step

timestep = 50 # You can look at an individual timestep. Lmk if you need a video. I have a seperate code for that.
# U Velocity 
# plt.subplot(1, 2, 1)
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(fig_width, fig_height), dpi=600,constrained_layout = True)
pc = uds_map['mesh2d_ucx'].isel(time=timestep).ugrid.plot(cmap=cmocean.cm.balance, add_colorbar=False, ax=axes[0], vmin=-1.5, vmax=1.5) 
# vmin and vmax need to be the same value (with the min negative) for max and min for white to be at the center
cbar = fig.colorbar(pc, ax=axes[0])
cbar.set_label('U-Velocity [m/s]', fontsize=font_size)
axes[0].set_aspect('equal')
axes[0].set_facecolor('whitesmoke')
axes[0].set_xlabel('Longitude', fontsize=font_size)
axes[0].set_ylabel('Latitude', fontsize=font_size)
axes[0].set_title(f'U-velocity, Timestep: {timestep}')


pc1 = uds_map['mesh2d_ucy'].isel(time=timestep).ugrid.plot(cmap=cmocean.cm.balance, add_colorbar=False, ax=axes[1], vmin=-1.5, vmax=1.5) 
# vmin and vmax need to be the same value (with the min negative) for max and min for white to be at the center
cbar = fig.colorbar(pc, ax=axes[1])
cbar.set_label('V-Velocity [m/s]', fontsize=font_size)
axes[1].set_aspect('equal')
axes[1].set_facecolor('whitesmoke')
axes[1].set_xlabel('Longitude', fontsize=font_size)
axes[1].set_ylabel('Latitude', fontsize=font_size)
axes[1].set_title(f'V-velocity, Timestep: {timestep}')
#fig.tight_layout()

# add color bar and adjust location
#fig.subplots_adjust(right=0.8)
#cbar_ax = fig.add_axes([0.85, 0.15, 0.05, 0.7])
#cbar = fig.colorbar(pc1, ax=axes[1])
#cbar.set_label('U- & V- Velocity [m/s]', fontsize=font_size)
#fig.colorbar(axes, cax=cbar_ax)
plt.savefig(r'C:\Users\sbrann5\OneDrive - Louisiana State University\May13_backup\Classwork\python_course\final_proj\figures\u_and_v_velocity.png', bbox_inches='tight') # use bbox to remove white space around image
plt.show()

In [ ]:
# rasterize image of velocity in domain and add velocity vectors instead of just each direction
raster_res=5000 # change to lower amount of arrows and smooth velocity magnitude field better
scale=7
max_vel=1.4

max_timestep=73 #found in attributes of the model
for timestep in range(1, max_timestep): # non-inclusive
    timestep = timestep 
    
    uds_quiv = uds_map.isel(time=timestep, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
    varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
    magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
    uds_quiv['magn'] = np.sqrt(uds_quiv[varn_ucx]**2+uds_quiv[varn_ucy]**2).assign_attrs(magn_attrs)
    raster_quiv = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

#plot
    fig,ax = plt.subplots(figsize=(fig_width, fig_height), dpi=600)
    pc = uds_quiv['magn'].ugrid.plot(cmap='jet',vmin=0, vmax=max_vel)
    raster_quiv.plot.quiver(x='mesh2d_face_x',y='mesh2d_face_y',u=varn_ucx,v=varn_ucy,color='black',scale=scale,add_guide=False)
    fig.tight_layout()
    cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    cbar.set_label('Velocity Magnitude [m/s]', fontsize=font_size)
    ax.set_aspect('equal')
    ax.set_facecolor('whitesmoke')
    ax.set_xlabel('Longitude', fontsize=font_size)
    ax.set_ylabel('Latitude', fontsize=font_size)
    time_date=uds_map['time'][timestep].values
    plt.title(f'Velocity at {time_date}') 

    plt.savefig(

In [ ]:
# Velocity gif over the duration of the simulation
max_timestep=73 #found in attributes of the model
for timestep in range(1,max_timestep): # non-inclusive
    timestep = timestep 
    time_date=uds_map['time'][timestep].values

# U Velocity 
# plt.subplot(1, 2, 1)
    fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(fig_width, fig_height), dpi=600,constrained_layout = True)
    pc = uds_map['mesh2d_ucx'].isel(time=timestep).ugrid.plot(cmap=cmocean.cm.balance, add_colorbar=False, ax=axes[0], vmin=-1.5, vmax=1.5) 
# vmin and vmax need to be the same value (with the min negative) for max and min for white to be at the center
    cbar = fig.colorbar(pc, ax=axes[0])
    cbar.set_label('U-Velocity [m/s]', fontsize=font_size)
    axes[0].set_aspect('equal')
    axes[0].set_facecolor('whitesmoke')
    axes[0].set_xlabel('Longitude', fontsize=font_size)
    axes[0].set_ylabel('Latitude', fontsize=font_size)
    axes[0].set_title(f'U-velocity at {time_date}')

    pc1 = uds_map['mesh2d_ucy'].isel(time=timestep).ugrid.plot(cmap=cmocean.cm.balance, add_colorbar=False, ax=axes[1], vmin=-1.5, vmax=1.5) 
# vmin and vmax need to be the same value (with the min negative) for max and min for white to be at the center
    cbar = fig.colorbar(pc, ax=axes[1])
    cbar.set_label('V-Velocity [m/s]', fontsize=font_size)
    axes[1].set_aspect('equal')
    axes[1].set_facecolor('whitesmoke')
    axes[1].set_xlabel('Longitude', fontsize=font_size)
    axes[1].set_ylabel('Latitude', fontsize=font_size)
    axes[1].set_title(f'V-velocity at {time_date}')
#fig.tight_layout()
    
    plt.savefig(f'C:/Users/sbrann5/OneDrive - Louisiana State University/May13_backup/Classwork/python_course/final_proj/figures/velocity_gif/u_and_v_velocity_{timestep}.png', bbox_inches='tight') # use bbox to remove white space around image


In [ ]:
# What is the water depth over time?

max_timestep=73 #found in attributes of the model
for timestep in range(1,max_timestep): # non-inclusive
    timestep = timestep 
    time_date=uds_map['time'][timestep].values
    
    fig, ax = plt.subplots(figsize=(fig_width, fig_height), dpi=600)
    pc = uds_map['mesh2d_waterdepth'].isel(time=timestep).ugrid.plot(cmap=cmocean.cm.ice_r, add_colorbar=False, ax=ax, vmin=0, vmax=50) #reversed order of colormap
    cbar = fig.colorbar(pc, ax=ax)
    cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
    cbar.set_label('Water Depth [m]', fontsize=font_size)
    ax.set_aspect('equal')
    ax.set_facecolor('whitesmoke')
    ax.set_xlabel('Longitude', fontsize=font_size)
    ax.set_ylabel('Latitude', fontsize=font_size)
    plt.title(f'Water depth at {time_date}') 
    plt.savefig(f'C:/Users/sbrann5/OneDrive - Louisiana State University/May13_backup/Classwork/python_course/final_proj/figures/water_depth/depth_{timestep}.png', bbox_inches='tight') # use bbox to remove white space around image
#plt.show()

In [ ]:
# Plot high tide and low tide together
# the tides are semi-diurnal (2 high 2 low per day), so high and low tide are offset by 6 hours
high_tide_timestep= 30
low_tide_timestep= 24
fig_height=5.5 # new so it doesn't stretch the figures
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(fig_width, fig_height), dpi=600,constrained_layout = True)
pc = uds_map_veg['mesh2d_waterdepth'].isel(time=high_tide_timestep).ugrid.plot(cmap=cmocean.cm.ice_r, add_colorbar=False, ax=axes[0,0], vmin=0, vmax=50) 
axes[0,0].set_facecolor('whitesmoke')
axes[0,0].set_xlabel('Longitude', fontsize=font_size)
axes[0,0].set_ylabel('Latitude', fontsize=font_size)
axes[0,0].set_title('High Tide Water Depth')

pc = uds_map['mesh2d_waterdepth'].isel(time=low_tide_timestep).ugrid.plot(cmap=cmocean.cm.ice_r, add_colorbar=False, ax=axes[0,1], vmin=0, vmax=50) 
cbar = fig.colorbar(pc, ax=axes[0,1])
cbar.set_label('Water Depth (m)', fontsize=font_size)
axes[0,1].set_facecolor('whitesmoke')
axes[0,1].set_xlabel('Longitude', fontsize=font_size)
axes[0,1].set_ylabel('Latitude', fontsize=font_size)
axes[0,1].set_title('Low Tide Water Depth')

uds_quiv = uds_map.isel(time=high_tide_timestep, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
uds_quiv['magn'] = np.sqrt(uds_quiv[varn_ucx]**2+uds_quiv[varn_ucy]**2).assign_attrs(magn_attrs)
raster_quiv = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

pc = uds_quiv['magn'].ugrid.plot(cmap='jet',vmin=0, vmax=max_vel, ax=axes[1,0], add_colorbar=False,)
#raster_quiv.plot.quiver(x='mesh2d_face_x',y='mesh2d_face_y',u=varn_ucx,v=varn_ucy,color='black',scale=scale,add_guide=False)
axes[1,0].set_facecolor('whitesmoke')
axes[1,0].set_xlabel('Longitude', fontsize=font_size)
axes[1,0].set_ylabel('Latitude', fontsize=font_size)
axes[1,0].set_title('High Tide Velocity')

uds_quiv = uds_map.isel(time=low_tide_timestep, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
uds_quiv['magn'] = np.sqrt(uds_quiv[varn_ucx]**2+uds_quiv[varn_ucy]**2).assign_attrs(magn_attrs)
raster_quiv = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

pc = uds_quiv['magn'].ugrid.plot(cmap='jet',vmin=0, vmax=max_vel, ax=axes[1,1], add_colorbar=False,) #add colors of velocity magnitude
#raster_quiv.plot.quiver(x='mesh2d_face_x',y='mesh2d_face_y',u=varn_ucx,v=varn_ucy,color='black',scale=scale,add_guide=False) #add arrows of velocity
#colorbar.remove()
cbar = fig.colorbar(pc, ax=axes[1,1])
cbar.ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
cbar.set_label('Velocity Magnitude [m/s]', fontsize=font_size)
axes[1,1].set_facecolor('whitesmoke')
axes[1,1].set_xlabel('Longitude', fontsize=font_size)
axes[1,1].set_ylabel('Latitude', fontsize=font_size)
axes[1,1].set_title('Low Tide Velocity')

plt.savefig('C:/Users/sbrann5/OneDrive - Louisiana State University/May13_backup/Classwork/python_course/final_proj/figures/vel_water_height_tides.png', bbox_inches='tight') # use bbox to remove white space around image


In [ ]:
# high-low tide water depth

difference= (uds_map_veg['mesh2d_waterdepth'].isel(time=high_tide_timestep))-(uds_map_veg['mesh2d_waterdepth'].isel(time=low_tide_timestep))
max_dif=np.max(difference).values
fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(fig_width, fig_height), dpi=600)
pc = difference.ugrid.plot(cmap=cmocean.cm.ice_r, add_colorbar=False, ax=axes, vmin=0, vmax=max_dif) 
axes.set_facecolor('whitesmoke')
axes.set_xlabel('Longitude', fontsize=font_size)
axes.set_ylabel('Latitude', fontsize=font_size)
axes.set_title('High Minus Low Tide Water Depth')
cbar = fig.colorbar(pc, ax=axes)
cbar.set_label('Water Depth Difference (m)', fontsize=font_size)
plt.savefig('C:/Users/sbrann5/OneDrive - Louisiana State University/May13_backup/Classwork/python_course/final_proj/figures/tidal_range.png', bbox_inches='tight') # use bbox to remove white space around image


In [ ]:
# difference in water velocity between high and low tide
# High - Low

uds_quiv_high = uds_map.isel(time=high_tide_timestep, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
uds_quiv_high['magn'] = np.sqrt(uds_quiv_high[varn_ucx]**2+uds_quiv_high[varn_ucy]**2).assign_attrs(magn_attrs)
raster_quiv_high = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

uds_quiv_low = uds_map.isel(time=low_tide_timestep, mesh2d_nLayers=-2, nmesh2d_layer=-2, missing_dims='ignore')
varn_ucx, varn_ucy = 'mesh2d_ucx', 'mesh2d_ucy'
magn_attrs = {'long_name':'velocity magnitude', 'units':'m/s'}
uds_quiv_low['magn'] = np.sqrt(uds_quiv_low[varn_ucx]**2+uds_quiv_low[varn_ucy]**2).assign_attrs(magn_attrs)
raster_quiv_low = dfmt.rasterize_ugrid(uds_quiv[[varn_ucx,varn_ucy]], resolution=raster_res)

difference = uds_quiv_high['magn']-uds_quiv_low['magn']
max_dif=np.max(difference).values

fig, axes = plt.subplots(nrows=1, ncols=1, figsize=(fig_width, fig_height), dpi=600)
pc = difference.ugrid.plot(cmap='jet',vmin=0, vmax=max_dif-0.3, ax=axes)
#raster_quiv.plot.quiver(x='mesh2d_face_x',y='mesh2d_face_y',u=varn_ucx,v=varn_ucy,color='black',scale=scale,add_guide=False)
axes.set_facecolor('whitesmoke')
axes.set_xlabel('Longitude', fontsize=font_size)
axes.set_ylabel('Latitude', fontsize=font_size)
axes.set_title('High - Low Tide Velocity')
plt.savefig('C:/Users/sbrann5/OneDrive - Louisiana State University/May13_backup/Classwork/python_course/final_proj/figures/dif_vel.png', bbox_inches='tight') # use bbox to remove white space around image

